In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def makeX(df):    #for making pandas dataframe to numpy ndarray and excluding column id
    return df.iloc[0:df.shape[0], 1:t_data.shape[1]].values
def makeY(df):
    return df.iloc[0:df.shape[0],1].values

In [7]:
def bar(X):             #making x x bar
    X = np.transpose(X)
    a = [np.full(X.shape[1],1)]
    X = np.append(X,a,0)
    return X

In [8]:
def ridgeReg(X,Y,lam):
    erra = []
    add = 0
    C = (np.dot(X,X.T) + (lam*np.identity(X.shape[0])))
    Cinv = np.linalg.inv(C)
    D = np.dot(X,Y)
    W = np.dot(Cinv, D) #
    for i in (0,X.shape[1]-1):
        xi = X.T[i]
        yi = Y[i]
        err = ((np.dot(xi.T,W) - yi) / (1 - (np.dot(np.dot(xi.T,Cinv),xi))))
        erra.append(err)
    for err in erra:
        add = add+(err**2)
    obj =  lam * (np.dot(W.T,W))+add
    rmse1 = np.sqrt(add)
    return W[0:W.shape[0]-1],W[W.shape[0]-1],obj,rmse1
    

In [9]:
def ridge(X,Y,lam):  # I wrote this function before writing combined function ridgeReg for some modularity for kaggle competition
    C = (np.dot(X,X.T) + (lam*np.identity(X.shape[0])))
    Cinv = np.linalg.inv(C)
    D = np.dot(X,Y)
    W = np.dot(Cinv, D)
    return W                              #, W[W.shape[0]-1]

In [10]:
def Loocv(X,Y,lam):
    erra = []
    add = 0
    C = (np.dot(X,X.T) + (lam*np.identity(X.shape[0])))
    Cinv = np.linalg.inv(C)
    W = ridge(X,Y,lam)
    for i in (0,X.shape[1]-1):
        xi = X.T[i]
        yi = Y[i]
        err = ((np.dot(xi.T,W) - yi) / (1 - (np.dot(np.dot(xi.T,Cinv),xi))))
        erra.append(err)
    for err in erra:
        add = add+(err**2)
    rmse = np.sqrt(add)
    return rmse

In [11]:
def predict(X,W):
    pred = np.dot(X.T,W)
    return pred

In [52]:
def rmse(vl, pred):
    add = 0
    err = vl-pred
    for val in err:
        add = add + (val ** 2)
    rmse = np.sqrt((add/1000))
    return rmse

    


In [24]:
lam = np.arange(0.1,1.1,0.1)

In [2]:
t_data = pd.read_csv("trainData.csv", header = None) 
tl_data = pd.read_csv("trainLabels.csv", header = None)

In [3]:
v_data = pd.read_csv("valData.csv",header = None)
vl_data = pd.read_csv("valLabels.csv",header = None)

In [18]:
Xtr = makeX(t_data)
Ytr = makeY(tl_data)

In [13]:
Valx = makeX(v_data)
Valy = makeY(vl_data)

In [20]:
Xtre = Valx[0:4000]
Ytre = Valy[0:4000]

In [14]:
ValX = Valx[4000:5000]
ValY = Valy[4000:5000] 

In [21]:
X = np.append(Xtr,Xtre,0)
Y = np.append(Ytr,Ytre,0)

In [22]:
X.shape

(9000, 3000)

In [23]:
X = bar(X)

In [25]:
ws = []
bs = []
es = []
ofs = []



In [26]:
for l in lam:
    a,b,c,d = ridgeReg(X,Y,l)
    ws.append(a)
    bs.append(b)
    ofs.append(c)
    es.append(d)

In [27]:
es

[1.5139027012585224,
 1.6095969764785583,
 1.6588612644099834,
 1.693596275240875,
 1.7220568202790607,
 1.7472704202778084,
 1.770528447643568,
 1.7924276369455605,
 1.8132584624093213,
 1.8331691254978606]

In [28]:
ValX = bar(ValX)

In [44]:
pred = []
for w in ws:
    pred.append(predict(np.append(w,[bs[0]],0),ValX))


In [46]:
len(pred)

10

In [58]:
err = []
for pre in pred:
    err.append(rmse(pre,ValY))
len(err)

10

In [59]:
err

[1.975066897538898,
 1.964146528230607,
 2.004962318568511,
 2.063436616645375,
 2.1269429081270346,
 2.190317599990449,
 2.2514054556245604,
 2.3093651632851344,
 2.363955284922542,
 2.415211804220919]

In [34]:
test0 = pd.read_csv("testData.csv",header= None)

In [35]:
test0 = makeX(test0)

In [36]:
test0 = bar(test0)

In [38]:
pred = predict(test0,np.append(ws[0],[bs[0]],0))

In [39]:
id4 = np.arange(0,5000)
id4.shape

(5000,)

In [40]:
df3 = pd.DataFrame()
df3['Id'] = id4
df3['Prediction'] = pred
df3.to_csv("predTestLabels.csv")

In [60]:
W = np.append(ws[1],[bs[1]],0)

In [70]:
sorted_w = np.argsort(W)

In [73]:
for w in sorted_w[2800:3000]:
    X_new = X.T[w]

In [75]:
X_new

array([0., 0., 0., ..., 0., 0., 1.])